# 0.1. imports

In [1]:
import bw2io
import bw2data
# type hints
from bw2io import SingleOutputEcospold2Importer

In [2]:
path_dir_datasets_ecoinvent31: str = '/Users/michaelweinold/data/data_raw/ecoinvent_3.1_cutoff_ecoSpold02/datasets'
path_dir_datasets_ecoinvent35: str = '/Users/michaelweinold/data/data_raw/ecoinvent_3.5_cutoff_ecoSpold02/datasets'
path_dir_datasets_ecoinvent38: str = '/Users/michaelweinold/data/data_raw/ecoinvent_3.8_cutoff_ecoSpold02/datasets'

# 1. `Brightway` setup
## 1.1. list all projects

In [3]:
bw2data.projects.current

'default'

In [4]:
bw2data.projects

Brightway2 projects manager with 4 objects:
	default
	ecoinvent31
	ecoinvent35
	ecoinvent38
Use `projects.report()` to get a report on all projects.

## 1.2. ensure no project is corrupted

if a project has been corrupted, simply delete it and re-populate it. \
🌐 compare [`bw2io` issue #118](https://github.com/brightway-lca/brightway2-data/issues/118)

In [21]:
try:
    bw2data.projects.delete_project(
        name = 'ecoinvent31',
        delete_dir = True
    )
except:
    pass

## 1.1. import `Ecoinvent 3.8`

"which `bw2io` version to use?"

compare the [Brightway mailing list](https://brightway.groups.io/g/development/message/154) and the [`bw2io` changelog](https://github.com/brightway-lca/brightway2-io/blob/main/CHANGES.md#085-2021-09-21): use `0.8.7`

In [6]:
bw2io.__version__

(0, 8, 7)

In [7]:
bw2data.projects.set_current("ecoinvent38")
bw2io.bw2setup()

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds
Title: Writing activities to SQLite3 database:
  Started: 01/06/2023 15:04:03
  Finished: 01/06/2023 15:04:03
  Total time elapsed: 00:00:00
  CPU %: 98.90
  Memory %: 0.53
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 0.61 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



In [8]:
importer_ei38: SingleOutputEcospold2Importer = bw2io.SingleOutputEcospold2Importer(
    dirpath = path_dir_datasets_ecoinvent38,
    db_name = 'ei38'
)

Extracting XML data from 19565 datasets
Extracted 19565 datasets in 14.79 seconds


In [9]:
importer_ei38.apply_strategies()
importer_ei38.statistics()

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: set_lognormal_loc_value
Applying strategy: convert_activity_parameters_to_list
App

(19565, 629959, 0)

## 1.2. import `Ecoinvent 3.5`

In [13]:
bw2data.projects.set_current("ecoinvent35")
bw2io.bw2setup()

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds
Title: Writing activities to SQLite3 database:
  Started: 01/06/2023 15:07:06
  Finished: 01/06/2023 15:07:06
  Total time elapsed: 00:00:00
  CPU %: 98.80
  Memory %: 18.80
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 0.63 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



In [19]:
importer_ei35: SingleOutputEcospold2Importer = bw2io.SingleOutputEcospold2Importer(
    dirpath = path_dir_datasets_ecoinvent35,
    db_name = 'ei35'
)

Extracting XML data from 16022 datasets
Extracted 16022 datasets in 16.81 seconds


In [20]:
importer_ei35.apply_strategies()
importer_ei35.statistics()

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: set_lognormal_loc_value
Applying strategy: convert_activity_parameters_to_list
App

(16022, 544735, 0)

## 1.3. import `Ecoinvent 3.1`

In [22]:
bw2data.projects.set_current("ecoinvent31")
bw2io.bw2setup()

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds
Title: Writing activities to SQLite3 database:
  Started: 01/06/2023 15:09:41
  Finished: 01/06/2023 15:09:41
  Total time elapsed: 00:00:00
  CPU %: 98.30
  Memory %: 25.08
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 0.61 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



In [23]:
importer_ei31: SingleOutputEcospold2Importer = bw2io.SingleOutputEcospold2Importer(
    dirpath = path_dir_datasets_ecoinvent31,
    db_name = 'ei31'
)

Extracting XML data from 11301 datasets
Extracted 11301 datasets in 12.50 seconds


In [24]:
importer_ei31.apply_strategies()
importer_ei31.statistics()

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: set_lognormal_loc_value
Applying strategy: convert_activity_parameters_to_list
App

(11301, 520932, 0)